In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [3]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.5)  


class EmailState(AgentState):
    email: str



agent = create_agent(
    #model="gpt-5-nano",
    model = model, 
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [4]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response immediately. Send the reply now in the same thread.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [5]:
from pprint import pprint

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='d3440cf5-43ce-4c23-8756-6c4f5c3898ef'),
              AIMessage(content=[{'type': 'text', 'text': 'I can do that. What would you like the reply to say?', 'extras': {'signature': 'CtwFAXLI2nzGahSh5VgXLP1TxDYeU1GTSOZOKvhCmq5uG5MXiO5ks6JAVRuoe+4kl4i+N+LGxmpkOmpmIFKLQ22AxlpYoz7tYVgIWm1XFblYd66XuS24xtb8LzTvv2TttesRaW8PlyE3GxoeHiVbbh8vTkBYG4mnRhMtaRuRQjppZm2/H3WI0pr4+bq0maPV8mOaz8SP5GCH7/XVu79rDOL1gyd/dLsCvpSo+iFJFobCIUG/uZn/CYAfM52zpqLlHY5EWoXKZ1D2selvuc2d3hA2qUmJSlQemLWtixmVBGcnJr/BLUuzkVfbmREG5RuajEjboHnDq3B9ytAaaRKnbXC/uul7PGt7VUGL++PpfA6esR664JLqXBqA8gj03QGcmsLYCQmJuPqMETvfngqnS25DL2+lDn3BUIO5+4fapgIBsEdAAX/WvULZ4tGNOKt7ywduDSu0hn6PWGQGfmh6oKx4IUa4mRu7ZclPYTf29K+VsfG1w5uxrTOFGS8Qhx7LiRvBtEt/D+9zlzjDa

In [ ]:
print(response['__interrupt__'])

In [ ]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

## Approve

In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume=
        {
            "decisions": [{"type": "approve"}]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='d3440cf5-43ce-4c23-8756-6c4f5c3898ef'),
              AIMessage(content=[{'type': 'text', 'text': 'I can do that. What would you like the reply to say?', 'extras': {'signature': 'CtwFAXLI2nzGahSh5VgXLP1TxDYeU1GTSOZOKvhCmq5uG5MXiO5ks6JAVRuoe+4kl4i+N+LGxmpkOmpmIFKLQ22AxlpYoz7tYVgIWm1XFblYd66XuS24xtb8LzTvv2TttesRaW8PlyE3GxoeHiVbbh8vTkBYG4mnRhMtaRuRQjppZm2/H3WI0pr4+bq0maPV8mOaz8SP5GCH7/XVu79rDOL1gyd/dLsCvpSo+iFJFobCIUG/uZn/CYAfM52zpqLlHY5EWoXKZ1D2selvuc2d3hA2qUmJSlQemLWtixmVBGcnJr/BLUuzkVfbmREG5RuajEjboHnDq3B9ytAaaRKnbXC/uul7PGt7VUGL++PpfA6esR664JLqXBqA8gj03QGcmsLYCQmJuPqMETvfngqnS25DL2+lDn3BUIO5+4fapgIBsEdAAX/WvULZ4tGNOKt7ywduDSu0hn6PWGQGfmh6oKx4IUa4mRu7ZclPYTf29K+VsfG1w5uxrTOFGS8Qhx7LiRvBtEt/D+9zlzjDa

## Reject

In [7]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='d3440cf5-43ce-4c23-8756-6c4f5c3898ef'),
              AIMessage(content=[{'type': 'text', 'text': 'I can do that. What would you like the reply to say?', 'extras': {'signature': 'CtwFAXLI2nzGahSh5VgXLP1TxDYeU1GTSOZOKvhCmq5uG5MXiO5ks6JAVRuoe+4kl4i+N+LGxmpkOmpmIFKLQ22AxlpYoz7tYVgIWm1XFblYd66XuS24xtb8LzTvv2TttesRaW8PlyE3GxoeHiVbbh8vTkBYG4mnRhMtaRuRQjppZm2/H3WI0pr4+bq0maPV8mOaz8SP5GCH7/XVu79rDOL1gyd/dLsCvpSo+iFJFobCIUG/uZn/CYAfM52zpqLlHY5EWoXKZ1D2selvuc2d3hA2qUmJSlQemLWtixmVBGcnJr/BLUuzkVfbmREG5RuajEjboHnDq3B9ytAaaRKnbXC/uul7PGt7VUGL++PpfA6esR664JLqXBqA8gj03QGcmsLYCQmJuPqMETvfngqnS25DL2+lDn3BUIO5+4fapgIBsEdAAX/WvULZ4tGNOKt7ywduDSu0hn6PWGQGfmh6oKx4IUa4mRu7ZclPYTf29K+VsfG1w5uxrTOFGS8Qhx7LiRvBtEt/D+9zlzjDa

In [ ]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

## Edit

In [10]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='d3440cf5-43ce-4c23-8756-6c4f5c3898ef'),
              AIMessage(content=[{'type': 'text', 'text': 'I can do that. What would you like the reply to say?', 'extras': {'signature': 'CtwFAXLI2nzGahSh5VgXLP1TxDYeU1GTSOZOKvhCmq5uG5MXiO5ks6JAVRuoe+4kl4i+N+LGxmpkOmpmIFKLQ22AxlpYoz7tYVgIWm1XFblYd66XuS24xtb8LzTvv2TttesRaW8PlyE3GxoeHiVbbh8vTkBYG4mnRhMtaRuRQjppZm2/H3WI0pr4+bq0maPV8mOaz8SP5GCH7/XVu79rDOL1gyd/dLsCvpSo+iFJFobCIUG/uZn/CYAfM52zpqLlHY5EWoXKZ1D2selvuc2d3hA2qUmJSlQemLWtixmVBGcnJr/BLUuzkVfbmREG5RuajEjboHnDq3B9ytAaaRKnbXC/uul7PGt7VUGL++PpfA6esR664JLqXBqA8gj03QGcmsLYCQmJuPqMETvfngqnS25DL2+lDn3BUIO5+4fapgIBsEdAAX/WvULZ4tGNOKt7ywduDSu0hn6PWGQGfmh6oKx4IUa4mRu7ZclPYTf29K+VsfG1w5uxrTOFGS8Qhx7LiRvBtEt/D+9zlzjDa